**Part 3.3: Saving and Loading a Keras Neural Network**
Complex neural networks will take a long time to fit/train. It is helpful to be able to save these neural networks so that they can be reloaded later. A reloaded neural network will not require retraining. Keras provides three formats for neural network saving.

YAML - Stores the neural network structure (no weights) in the YAML file format.
JSON - Stores the neural network structure (no weights) in the JSON file format.
HDF5 - Stores the complete neural network (with weights) in the HDF5 file format. Do not confuse HDF5 with HDFS. They are different. We do not use HDFS in this class.
Usually you will want to save in HDF5.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

save_path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression


In [2]:
x

array([[  8. , 307. , 130. , ...,  12. ,  70. ,   1. ],
       [  8. , 350. , 165. , ...,  11.5,  70. ,   1. ],
       [  8. , 318. , 150. , ...,  11. ,  70. ,   1. ],
       ...,
       [  4. , 135. ,  84. , ...,  11.6,  82. ,   1. ],
       [  4. , 120. ,  79. , ...,  18.6,  82. ,   1. ],
       [  4. , 119. ,  82. , ...,  19.4,  82. ,   1. ]])

In [3]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 25. ,
       19. , 16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. ,
       13. , 18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. ,
       27. , 26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. ,
       17. , 11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. ,
       22. , 21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. ,
       14. , 15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. ,
       18. , 18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. ,
       22. , 18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. ,
       15. , 24. , 20. , 11. , 20. , 21. , 19. , 15. , 31. , 26. , 32. ,
       25. , 16. , 16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. ,
       26. , 31. , 32. , 28. , 24. , 26. , 24. , 26

In [5]:
# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score (RMSE): {score}")

# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(save_path,"network.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"network.h5"))

Epoch 1/100
13/13 - 0s - loss: 12662.2314
Epoch 2/100
13/13 - 0s - loss: 3354.4729
Epoch 3/100
13/13 - 0s - loss: 938.9813
Epoch 4/100
13/13 - 0s - loss: 513.4806
Epoch 5/100
13/13 - 0s - loss: 454.8716
Epoch 6/100
13/13 - 0s - loss: 431.9193
Epoch 7/100
13/13 - 0s - loss: 418.1751
Epoch 8/100
13/13 - 0s - loss: 398.4967
Epoch 9/100
13/13 - 0s - loss: 385.0410
Epoch 10/100
13/13 - 0s - loss: 375.2057
Epoch 11/100
13/13 - 0s - loss: 367.8313
Epoch 12/100
13/13 - 0s - loss: 354.4082
Epoch 13/100
13/13 - 0s - loss: 345.3369
Epoch 14/100
13/13 - 0s - loss: 330.2050
Epoch 15/100
13/13 - 0s - loss: 319.6284
Epoch 16/100
13/13 - 0s - loss: 309.5263
Epoch 17/100
13/13 - 0s - loss: 297.6205
Epoch 18/100
13/13 - 0s - loss: 288.2582
Epoch 19/100
13/13 - 0s - loss: 279.6165
Epoch 20/100
13/13 - 0s - loss: 273.2843
Epoch 21/100
13/13 - 0s - loss: 256.9766
Epoch 22/100
13/13 - 0s - loss: 246.1835
Epoch 23/100
13/13 - 0s - loss: 238.0937
Epoch 24/100
13/13 - 0s - loss: 231.9507
Epoch 25/100
13/13 - 0

In [6]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 6.736348665674522
